In [10]:
from collections import defaultdict
from collections import Counter
import operator
import numpy as np
import pandas as pd
import csv
import time


td = "/Users/sreenathgv/Learning/Masters/Social_Dynamics_Lab/Reddit_populism/td_data_v2.csv"
sfp = "/Users/sreenathgv/Learning/Masters/Social_Dynamics_Lab/Reddit_populism/sfp_data_v2.csv"
neither = "/Users/sreenathgv/Learning/Masters/Social_Dynamics_Lab/Reddit_populism/neither_sample.csv"
coding = "/Users/sreenathgv/Learning/Masters/Social_Dynamics_Lab/Reddit_populism/political_coding_fix.csv"

### The_Donald

In [11]:
political_coding = pd.read_csv(coding)
with open(td) as f:
    reader = csv.reader(f) 
    first_row = True  
    
    # author_history_td : dict {'author':[(subreddit, timestamp), ...................,(subreddit, timestamp)}
    author_history_td = {}
    for row in reader:
        if first_row:
            first_row = False
            continue   
        author = row[0]
        subreddit = row[1]
        created_utc = row[3]
        
        if author in author_history_td.keys():
            author_history_td[author].append((subreddit, int(created_utc)))
        else:
            author_history_td[author] = [(subreddit, int(created_utc))]       

Here we compute the cutoff dates for each author and subset the data set accordingly.


In [12]:
author_dt_range = defaultdict()
author_hist_td_subset = defaultdict()
unique_subreddits = set()

# For each author finding the UTC cutoff: [first post in r/The_Donald]
for auth in author_history_td:
    td_lst = []
    for item in author_history_td[auth]:
        if item[0] == 'The_Donald':
            td_lst.append(item)
        else:
            unique_subreddits.add(item[0])
            
    end_dt = min(td_lst, key=operator.itemgetter(1))[1]
    author_dt_range[auth] = end_dt

# Subset the data to include only posts before UTC
for auth in author_history_td:
    author_hist_td_subset[auth] = [x for x in author_history_td[auth] if (x[1] < author_dt_range[auth])]

unique_authors_td = list(author_dt_range.keys())
author_idx = defaultdict()
subreddit_idx = defaultdict()
author_idx = {k: v for v, k in enumerate(list(author_dt_range.keys()))}
subreddit_idx = {k: v for v, k in enumerate(list(unique_subreddits))}

1. From the data subset above, for each author we compute the count of posts in each subreddit(tp_part_matrix). 
2. To find participation proportion we divide the counts by total number of posts made by the author. Both count as well participation scores are represented as numpy arrays.
3. Final participation proportion for a subreddit, is the average proportion across all authors


In [14]:
td_part_matrix = np.zeros((len(author_idx),len(subreddit_idx)))
for auth in author_hist_td_subset:
    aut_idx = author_idx[auth]
    for item in author_hist_td_subset[auth]:
        sub_idx = subreddit_idx[item[0]]
        td_part_matrix[aut_idx][sub_idx] += 1

td_part_prop = np.copy(td_part_matrix)
td_part_prop = (td_part_prop.T/(np.sum(td_part_prop, axis =1))).T
td_sub_participation = np.mean(td_part_prop, axis =0)

subs = []
for i, item in enumerate(subreddit_idx.keys()):
    subs.append((item,round(td_sub_participation[subreddit_idx[item]]*100,2)))
td_sorted_subs = sorted(subs,key=operator.itemgetter(1),reverse = True)
td_participation = pd.DataFrame(td_sorted_subs, columns=['subreddit', 'participation_prop'])

In [27]:
td_participation_ful = pd.merge(td_participation, political_coding, on='subreddit', how='left')
td_participation_ful.head(20)

,subreddit,participation_prop,political,populist,partisan_lean
0,AskReddit,7.52,NaN,NaN,NaN
1,politics,4.65,Yes,No,Left
2,funny,3.05,NaN,NaN,NaN
3,pics,2.71,NaN,NaN,NaN
4,SandersForPresident,2.61,Yes,Yes,Left
5,AdviceAnimals,2.37,NaN,NaN,NaN
6,worldnews,2.00,NaN,NaN,NaN
7,leagueoflegends,1.94,NaN,NaN,NaN
8,videos,1.74,NaN,NaN,NaN
9,news,1.72,NaN,NaN,NaN


In [ ]:
td_participation_ful.to_csv("The_Donald_participation.csv")

### SFP

In [28]:
with open(sfp) as f:
    reader = csv.reader(f) 
    first_row = True  
    
    # author_history_sfp : dict {'author':[(subreddit, timestamp), ...................,(subreddit, timestamp)}
    author_history_sfp = {}
    for row in reader:
        if first_row:
            first_row = False
            continue   
        author = row[0]
        subreddit = row[1]
        created_utc = row[3]
        
        if author in author_history_sfp.keys():
            author_history_sfp[author].append((subreddit, int(created_utc)))
        else:
            author_history_sfp[author] = [(subreddit, int(created_utc))]       

In [29]:
author_dt_range = defaultdict()
author_hist_sfp_subset = defaultdict()
unique_subreddits = set()

# For each author finding the UTC cutoff: [first post in r/The_Donald]
for auth in author_history_sfp:
    sfp_lst = []
    for item in author_history_sfp[auth]:
        if item[0] == 'SandersForPresident':
            sfp_lst.append(item)
        else:
            unique_subreddits.add(item[0])
            
    end_dt = min(sfp_lst, key=operator.itemgetter(1))[1]
    author_dt_range[auth] = end_dt

# Subset the data to include only posts before UTC
for auth in author_history_sfp:
    author_hist_sfp_subset[auth] = [x for x in author_history_sfp[auth] if (x[1] < author_dt_range[auth])]

unique_authors_sfp = list(author_dt_range.keys())
author_idx = defaultdict()
subreddit_idx = defaultdict()
author_idx = {k: v for v, k in enumerate(list(author_dt_range.keys()))}
subreddit_idx = {k: v for v, k in enumerate(list(unique_subreddits))}

In [30]:
sfp_part_matrix = np.zeros((len(author_idx),len(subreddit_idx)))

for auth in author_hist_sfp_subset:
    aut_idx = author_idx[auth]
    for item in author_hist_sfp_subset[auth]:
        sub_idx = subreddit_idx[item[0]]
        sfp_part_matrix[aut_idx][sub_idx] += 1


sfp_part_prop = np.copy(sfp_part_matrix)
sfp_part_prop = (sfp_part_prop.T/(np.sum(sfp_part_prop, axis =1))).T
sfp_sub_participation = np.mean(sfp_part_prop, axis =0)


subs = []
for i, item in enumerate(subreddit_idx.keys()):
    subs.append((item,round(sfp_sub_participation[subreddit_idx[item]]*100,2)))
sfp_sorted_subs = sorted(subs,key=operator.itemgetter(1),reverse = True)

sfp_participation = pd.DataFrame(sfp_sorted_subs, columns=['subreddit', 'participation_prop'])

In [32]:
sfp_participation_ful = pd.merge(sfp_participation, political_coding, on='subreddit', how='left')
sfp_participation_ful.head(20)


,subreddit,participation_prop,political,populist,partisan_lean
0,AskReddit,9.27,NaN,NaN,NaN
1,funny,2.78,NaN,NaN,NaN
2,pics,2.53,NaN,NaN,NaN
3,politics,2.27,Yes,No,Left
4,AdviceAnimals,1.88,NaN,NaN,NaN
5,leagueoflegends,1.52,NaN,NaN,NaN
6,videos,1.50,NaN,NaN,NaN
7,worldnews,1.50,NaN,NaN,NaN
8,WTF,1.40,NaN,NaN,NaN
9,gaming,1.35,NaN,NaN,NaN


In [33]:
sfp_participation_ful.to_csv("Sanders_for_President_participation.csv")

### Neither

In [34]:
political_coding = pd.read_csv(coding)
with open(neither) as f:
    reader = csv.reader(f) 
    first_row = True  
    
    # author_history_nt : dict {'author':[(subreddit, timestamp), ...................,(subreddit, timestamp)}
    author_history_nt = {}
    for row in reader:
        if first_row:
            first_row = False
            continue   
        author = row[0]
        subreddit = row[1]
        created_utc = row[3]
        
        if author in author_history_nt.keys():
            author_history_nt[author].append((subreddit, int(created_utc)))
        else:
            author_history_nt[author] = [(subreddit, int(created_utc))]       

In [35]:
author_hist_nt_subset = defaultdict()
unique_subreddits = set()
unique_authors_nt = set()

for auth in author_history_nt:
    unique_authors_nt.add(auth)
    for item in author_history_nt[auth]:
        unique_subreddits.add(item[0])

unique_authors_nt = list(unique_authors_nt)
# Subset the data to include only posts before UTC
for auth in author_history_nt:
    author_hist_nt_subset[auth] = [x for x in author_history_nt[auth] if (x[1] < 1514764799)]

author_idx = defaultdict()
subreddit_idx = defaultdict()
author_idx = {k: v for v, k in enumerate(list(unique_authors_nt))}
subreddit_idx = {k: v for v, k in enumerate(list(unique_subreddits))}

In [36]:
nt_part_matrix = np.zeros((len(author_idx),len(subreddit_idx)))
for auth in author_hist_nt_subset:
    aut_idx = author_idx[auth]
    for item in author_hist_nt_subset[auth]:
        sub_idx = subreddit_idx[item[0]]
        nt_part_matrix[aut_idx][sub_idx] += 1

nt_part_prop = np.copy(nt_part_matrix)
nt_part_prop = (nt_part_prop.T/(np.sum(nt_part_prop, axis =1))).T
nt_sub_participation = np.mean(nt_part_prop, axis =0)

subs = []
for i, item in enumerate(subreddit_idx.keys()):
    subs.append((item,round(nt_sub_participation[subreddit_idx[item]]*100,2)))
nt_sorted_subs = sorted(subs,key=operator.itemgetter(1),reverse = True)
nt_participation = pd.DataFrame(nt_sorted_subs, columns=['subreddit', 'participation_prop'])

In [37]:
nt_participation_ful = pd.merge(nt_participation, political_coding, on='subreddit', how='left')
nt_participation_ful.head(20)

,subreddit,participation_prop,political,populist,partisan_lean
0,AskReddit,8.79,NaN,NaN,NaN
1,funny,2.18,NaN,NaN,NaN
2,pics,1.98,NaN,NaN,NaN
3,leagueoflegends,1.93,NaN,NaN,NaN
4,gaming,1.33,NaN,NaN,NaN
5,WTF,1.25,NaN,NaN,NaN
6,AdviceAnimals,1.15,NaN,NaN,NaN
7,videos,1.11,NaN,NaN,NaN
8,politics,1.01,Yes,No,Left
9,todayilearned,0.98,NaN,NaN,NaN


In [38]:
overlap_TD_SFP = list(set(unique_authors_sfp) & set(unique_authors_td))
perc_TD_in_SFP = len(overlap_TD_SFP)*100/len(unique_authors_td)
perc_TD_in_SFP

perc_SFP_in_TD = len(overlap_TD_SFP)*100/len(unique_authors_sfp)
perc_SFP_in_TD

0.0

### Policital, Populist, Partisan Lean Status

In [51]:
print('\033[1m  \nThe_Donald: Average paricipation proportion  \033[0m')

print("\n Political:")
print(td_participation_ful.groupby('political', as_index=False)['participation_prop'].mean())

print("\n Populist:")
print(td_participation_ful.groupby('populist', as_index=False)['participation_prop'].mean())

print("\n partisan_lean:")
print(td_participation_ful.groupby('partisan_lean', as_index=False)['participation_prop'].mean())

  
The_Donald: Average paricipation proportion  

 Political:
  political  participation_prop
0       Yes            0.009474

 Populist:
  populist  participation_prop
0       No            0.022979
1  Unknown            0.000667
2      Yes            0.005500

 partisan_lean:
  partisan_lean  participation_prop
0      Centrist            0.000000
1          Left            0.015217
2         Right            0.007551
3       Unknown            0.000714


In [53]:
print('\033[1m  \n Sanders_For_President : Average paricipation proportion  \033[0m')

print("\n Political:")
print(sfp_participation_ful.groupby('political', as_index=False)['participation_prop'].mean())

print("\n Populist:")
print(sfp_participation_ful.groupby('populist', as_index=False)['participation_prop'].mean())

print("\n partisan_lean:")
print(sfp_participation_ful.groupby('partisan_lean', as_index=False)['participation_prop'].mean())

  
 Sanders_For_President : Average paricipation proportion  

 Political:
  political  participation_prop
0       Yes            0.012833

 Populist:
  populist  participation_prop
0       No            0.044464
1  Unknown            0.002090
2      Yes            0.006647

 partisan_lean:
  partisan_lean  participation_prop
0      Centrist            0.000000
1          Left            0.015872
2         Right            0.012933
3       Unknown            0.001463
